In [21]:
import numpy as np
import tensorflow as tf
import keras 
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM

# From the keras lstm_text_generation.py Github example
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

def gen_rnn_data(n, filename):
    all_text = ""
    with open(filename) as inp:
        for line in inp:
            if len(line.split()) > 1:
                all_text += line[:-1].lower()
    seqs = []
    for i in range(40, len(all_text), n):
        seqs.append(all_text[i-40:i+1])
    chars = sorted(list(set('\n'.join(seqs))))
    char_2_index = dict((c, i) for i, c in enumerate(chars))
    index_2_char = dict((i, c) for i, c in enumerate(chars))
    return seqs, char_2_index, index_2_char 


def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()


_, mapping_c2i, mapping_i2c = gen_rnn_data(10, '../data/shakespeare.txt')
print(mapping_c2i)

def train_model(seq_skip, n_nodes, n_epochs, model_filename):
    lines, mapping_c2i, mapping_i2c = gen_rnn_data(seq_skip, '../data/shakespeare.txt')
    all_text = '\n'.join(lines)

    seqs = []
    for line in lines:
        # Replace string with encoding of each character
        encoded_seq = [mapping_c2i[char] for char in line]
        # Replace list of sequences with list of encoded sequences
        seqs.append(encoded_seq)

    # Number of distinct characters
    num_chars = len(mapping_c2i)
    print(num_chars)
    # Splitting into input strings and output character
    seqs = np.array(seqs)
    X, y = seqs[:, :-1], seqs[:, -1]

    # one-hot encoding
    seqs = [to_categorical(x, num_classes = num_chars) for x in X]
    X = np.array(seqs)
    y = to_categorical(y, num_classes=num_chars)

    # Model

    model = Sequential()
    model.add(LSTM(n_nodes, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dense(num_chars, activation='softmax'))
    print(model.summary())

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X, y, epochs=n_epochs, verbose=1)

    model.save(model_filename)

{'\n': 0, ' ': 1, '!': 2, "'": 3, '(': 4, ')': 5, ',': 6, '-': 7, '.': 8, ':': 9, ';': 10, '?': 11, 'a': 12, 'b': 13, 'c': 14, 'd': 15, 'e': 16, 'f': 17, 'g': 18, 'h': 19, 'i': 20, 'j': 21, 'k': 22, 'l': 23, 'm': 24, 'n': 25, 'o': 26, 'p': 27, 'q': 28, 'r': 29, 's': 30, 't': 31, 'u': 32, 'v': 33, 'w': 34, 'x': 35, 'y': 36, 'z': 37}


In [22]:
train_model(10, 200, 30, 'test_model_2.h')
#train_model(15, 250, 25, 'hip_hop2.h')

38
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 200)               191200    
_________________________________________________________________
dense_5 (Dense)              (None, 38)                7638      
Total params: 198,838
Trainable params: 198,838
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/30
9210/9210 [==============================] - 18s 2ms/step - loss: 2.9275 - acc: 0.1779
Epoch 2/30
9210/9210 [==============================] - 18s 2ms/step - loss: 2.5783 - acc: 0.2771
Epoch 3/30
9210/9210 [==============================] - 19s 2ms/step - loss: 2.3839 - acc: 0.3054
Epoch 4/30
9210/9210 [==============================] - 18s 2ms/step - loss: 2.2858 - acc: 0.3366
Epoch 5/30
9210/9210 [==============================] - 17s 2ms/step - loss: 2.2017 - acc: 0.3545
Epoch 6/30
9210/9210 [============

In [24]:
# Inspired by (but not copied from) the keras example Github
def sample(predictions, temperature=1.0):
    predictions = np.asarray(predictions).astype('float64')
    predictions = np.log(predictions) / temperature
    e_predictions = np.exp(predictions)
    predictions = e_predictions / np.sum(e_predictions)
    probs = np.random.multinomial(1, predictions[0], 1)
    return np.argmax(probs)

def generate_text(model_filename, seed_text, n_chars, temp):
    model = load_model(model_filename)
    for _ in range(n_chars):
        temp_seed = seed_text[-40:]
        encoded = [mapping_c2i[char] for char in temp_seed]
        encoded = np.array(to_categorical(encoded, num_classes=len(mapping_c2i)))
        encoded = np.reshape(encoded, (1, encoded.shape[0], encoded.shape[1]))

        preds = model.predict(encoded, verbose=0)
        ind = sample(preds, temperature=temp)
        seed_text += mapping_i2c[ind]
    return seed_text

def generate_poem(model_filename, seed_text, temp):
    poem_text = generate_text(model_filename, seed_text, 520, temp)
    lines = [poem_text[i:i+40] for i in range(0, 520, 40)]
    poem = ''
    for line in lines:
        if '\n' in line:
            line = line.replace('\n', '')
        poem += line
        poem += '\n'
    return poem

In [25]:
from keras.models import load_model


print("Poem with 1.5 temperature")
print(generate_poem('test_model_2.h', "shall i compare thee to a summer's day?\n", 1.5))

print("Poem with 0.75 temperature")
print(generate_poem('test_model_2.h', "shall i compare thee to a summer's day?\n", 0.75))

print("Poem with 0.25 temperature")
print(generate_poem('test_model_2.h', "shall i compare thee to a summer's day?\n", 0.25))

Poem with 1.5 temperature
shall i compare thee to a summer's day?
owexfadmowh tholst fen and me,beit ghatt
 doth minds wpei hrave doon hase,whish i
 jove that sichtwer breentulnenswed care
chidg tongst fy love frevevt:whtt tine i
n knyer qaies bo thince siccadgich offrr
were,se, and mush chagh dus and cinesent
ss on beains,ho gavtszo yos rrss.isweje 
bo on ers noued so my trees from ase mos
trrss wrensh bid,s by thy owhe, ast to t
he'r pruistrese,what basbead shoull tome
 yousale batiestinghdas ors flifere butk
shtseichadsed d chack laks goed com,thic

Poem with 0.75 temperature
shall i compare thee to a summer's day?
y ailftr owres chouts thouss swrets doos
s wrogh dut in sher, on whish whrt in do
st tongadds llasing of al nore's with su
alf love's batt hing in that reatung tha
t looss songs fross then shate ofirgore'
s brensere nat hase but of my lever no s
hat pome,fofloth nat inqusen ene,d seefe
nghy siuld beady hot hish dorg thou sove
 foof faald bo holr wnot hasco sires ond
 for 

In [26]:
print("Poem with 0.15 temperature")
print(generate_poem('test_model_2.h', "shall i compare thee to a summer's day?\n", 0.15))

print("Poem with 0.1 temperature")
print(generate_poem('test_model_2.h', "shall i compare thee to a summer's day?\n", 0.1))

print("Poem with 0.05 temperature")
print(generate_poem('test_model_2.h', "shall i compare thee to a summer's day?\n", 0.05))

Poem with 0.15 temperature
shall i compare thee to a summer's day?
y aillt, that sthen the bucuiferis bady 
ar my reasurgase and and,dos best it sim
f,  for in entir fainothat i const ferer
 wowhate,that thou not praine,what is my
 soll no saire,whs io thy heart thee my 
reairwhat sumend sight be bute si love, 
now hours sate aich of floode,that sicht
 fis love's batt his hien, sight is bain
y bate,a dosh mend, rwhimet thy sey wath
 stulle's neded mo badt pom my fove that
 suclove stolle, with hou love not sairs
 of my wrich sight for love, hot hish ba

Poem with 0.1 temperature
shall i compare thee to a summer's day?
y aillt, that sthen the bucuie, o have i
n deersto say woth my love so love, wnst
 bost if thas beat thy shad,a tuee me lo
ve youl nith, simmor that wire shiin eys
 and doth fishad i has pare,and fair the
 comerdst thy swail, you govet,  sway wh
e love to sout wot stares of not prom th
en braine,than i succt love sueles:by th
ou com't to se.wrich shith pyrist dosh t
hress